In [1]:
# -*- coding: utf-8 -*-
"""
InfoPool (single-period) — batch 20 τ scenarios with asset-level exports.

What this script does
---------------------
1) Build Ω⁺/Ω⁻ from market data (yfinance), your risk/utility model, and set up Xpress.
2) For k = 0..20 (number of shared assets by Account 0):
   - Construct τ^(k): start from your base τ (Acct0 row = all zeros), then flip the first k assets in row-0 to 1.
   - Solve the InfoPool optimization with that τ.
   - Export:
     (a) Per-account summary: Revenue, Risk, TransactionCost, NetUtility, SelfImpact(sum over assets).
     (b) Asset-level trade details per account: w⁺, w⁻, q⁺=τ·w⁺, q⁻=τ·w⁻, s_{j,i} (per-asset self-impact, diag-only).
     (c) Per-asset pool table: T⁺, T⁻, pooled_cost_diag (the rebate fund A_i), participants set P_i.
   - All CSVs are named with suffix `_tau_k{K}.csv` to feed your multi-period rebate code later.

Also exports invariant inputs once: Omega_plus.csv, Omega_minus.csv, tickers.csv, budgets.csv.
"""

import os
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import xpress as xp

# -----------------------------
# 0) Paths & switches
# -----------------------------
OUTDIR = "infopool_batch_outputs"
os.makedirs(OUTDIR, exist_ok=True)

SAVE_MATRICES_ONCE = True      # write Ω⁺/Ω⁻ etc. once
ANNUAL_TRADING_DAYS = 252

# -----------------------------
# 1) Market data & Ω-construction (unchanged in batch)
# -----------------------------
xp.init("C:/Users/s2667242/AppData/Local/anaconda3/Lib/site-packages/xpress/license/xpauth.xpr")

tickers = [
    "AAPL","MSFT","GOOGL","AMZN","META","TSLA","JNJ","JPM","V","PG",
    "NVDA","DIS","UNH","HD","MA","PFE","BAC","KO","XOM","CVX"
]
d = len(tickers)

start_date = "2020-01-01"
end_date   = "2025-07-24"

data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False, progress=False)
prices_adj = data["Adj Close"]
prices_raw = data["Close"]
volumes    = data["Volume"]

returns = prices_adj.pct_change().dropna()
mean_daily_returns = returns.mean()
cov_matrix         = returns.cov()

sigma_all  = returns.std(axis=0).values
up_returns   = returns.clip(lower=0)
down_returns = (-returns).clip(lower=0)
sigma_plus   = up_returns.std(axis=0).values
sigma_minus  = down_returns.std(axis=0).values

dollar_vol = (prices_raw.reindex_like(volumes) * volumes).loc[returns.index]
Vbar = dollar_vol.mean(axis=0).values  # shape(d,)

def winsorize_ser(s: pd.Series, q=0.995):
    lo, hi = s.quantile(1-q), s.quantile(q)
    return s.clip(lower=lo, upper=hi)

def estimate_kappa_from_illiq(prices_raw: pd.DataFrame,
                              volumes: pd.DataFrame,
                              returns: pd.DataFrame,
                              sigma_vec, side="both"):
    px_raw = prices_raw.loc[returns.index]
    vol    = volumes.loc[returns.index]
    dollar_vol = (px_raw * vol).replace(0, np.nan)
    illiq = (returns.abs() / dollar_vol).replace([np.inf, -np.inf], np.nan)

    if side == "plus":
        mask = (returns > 0)
    elif side == "minus":
        mask = (returns < 0)
    else:
        mask = pd.DataFrame(True, index=returns.index, columns=returns.columns)

    illiq_med = illiq.where(mask).median(axis=0, skipna=True)
    illiq_med = winsorize_ser(illiq_med, q=0.995)
    Vbar_local = (px_raw * vol).mean(axis=0)
    sigma = pd.Series(sigma_vec, index=returns.columns)
    kappa = (illiq_med * Vbar_local / sigma).replace([np.inf, -np.inf], np.nan)
    kappa = kappa.clip(lower=0.01, upper=100.0)
    return kappa.values

def gershgorin_delta_max(gamma: np.ndarray, corr: np.ndarray):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    row_sum_off = np.where(row_sum_off <= 1e-12, np.inf, row_sum_off)
    return float(np.min(gamma / row_sum_off))

def delta_from_ratio(gamma: np.ndarray, corr: np.ndarray, rho=0.3, cap_ratio=0.9):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    factor = row_sum_off / gamma
    mean_factor = np.mean(factor[np.isfinite(factor) & (factor > 0)])
    delta_star = rho / mean_factor if (mean_factor is not None and mean_factor > 0) else 0.0
    delta_max  = gershgorin_delta_max(gamma, corr)
    return float(min(delta_star, cap_ratio * delta_max)), float(delta_max)

kappa_plus_hat  = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_plus,  side="plus")
kappa_minus_hat = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_minus, side="minus")
gamma_plus  = kappa_plus_hat  * sigma_plus  / Vbar
gamma_minus = kappa_minus_hat * sigma_minus / Vbar

corr = cov_matrix.values / np.outer(sigma_all, sigma_all)
delta_plus,  _ = delta_from_ratio(gamma_plus,  corr, rho=0.3, cap_ratio=0.9)
delta_minus, _ = delta_from_ratio(gamma_minus, corr, rho=0.3, cap_ratio=0.9)

Gp = np.sqrt(np.outer(gamma_plus,  gamma_plus))
Gm = np.sqrt(np.outer(gamma_minus, gamma_minus))
Omega_plus  = np.diag(gamma_plus)  + delta_plus  * (Gp * corr)
Omega_minus = np.diag(gamma_minus) + delta_minus * (Gm * corr)

xi = 0.10
target_bps = 100
c = target_bps / 1e4
s_cand_p = 2*c / np.maximum(gamma_plus  * xi * Vbar, 1e-30)
s_cand_m = 2*c / np.maximum(gamma_minus * xi * Vbar, 1e-30)
s_omega  = float(np.median(np.concatenate([s_cand_p, s_cand_m])))
Omega_plus  = s_omega * Omega_plus
Omega_minus = s_omega * Omega_minus

# -----------------------------
# 2) Investor preferences & budgets (unchanged in batch)
# -----------------------------
N = 10
trading_days = ANNUAL_TRADING_DAYS

mu_vec     = mean_daily_returns.values
Sigma_mat  = cov_matrix.values
mu_annual   = mu_vec * trading_days
Sigma_annual= Sigma_mat * trading_days
mu_list    = [mu_annual.copy()  for _ in range(N)]
Sigma_list = [Sigma_annual.copy() for _ in range(N)]

gamma_list = [
    0.49671415, 0.64768854, 0.76743473, 0.54256004, 0.24196227,
    0.31424733, 0.06752820, 0.11092259, 0.37569802, 0.82254491
]
budgets = [1e6, 5e5, 8e5, 1.2e6, 6e5, 9e5, 7e5, 4e5, 1.5e6, 1e6]

alpha_risk = 0.5
w_pct0 = np.full((N, d), 1.0/d)
risk_proto = np.array([
    0.5 * gamma_list[n] * (budgets[n]**2) * (w_pct0[n] @ Sigma_list[n] @ w_pct0[n])
    for n in range(N)
])
rev_est = np.array([budgets[n] * (mu_list[n] @ w_pct0[n]) for n in range(N)])
s = (alpha_risk * max(rev_est.mean(), 1e-12)) / max(risk_proto.mean(), 1e-12)
theta_list = (s * np.array(gamma_list)).tolist()

# Initial holdings (deterministic)
INIT_HOLDINGS_PATH = os.path.join(OUTDIR, "initial_holdings.csv")
INIT_SEED = 12345
def _dirichlet_like_from_hash(tickers, budgets, seed=INIT_SEED):
    Nn, dd = len(budgets), len(tickers)
    W = np.zeros((Nn, dd), dtype=float)
    for n in range(Nn):
        e = np.empty(dd, dtype=float)
        for j, t in enumerate(tickers):
            key = f"{seed}|acct={n}|ticker={t}".encode()
            h = hashlib.sha256(key).digest()
            u64 = int.from_bytes(h[:8], "big")
            u = (u64 + 0.5) / 2**64
            u = min(max(u, 1e-12), 1-1e-12)
            e[j] = -np.log(u)
        w = e / e.sum()
        W[n, :] = w
    B = np.asarray(budgets, dtype=float)[:, None]
    return W * B

def _load_or_make_initial_holdings(tickers, budgets, path=INIT_HOLDINGS_PATH, seed=INIT_SEED):
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0)
        ok_shape = (df.shape[0] == len(budgets)) and (df.shape[1] == len(tickers))
        ok_cols  = list(df.columns) == list(tickers)
        if ok_shape and ok_cols:
            return df.values
    arr = _dirichlet_like_from_hash(tickers, budgets, seed=seed)
    df = pd.DataFrame(arr, index=[f"Acct {i}" for i in range(len(budgets))], columns=tickers)
    df.to_csv(path, float_format="%.6f")
    return arr

old_holdings = _load_or_make_initial_holdings(tickers, budgets)

# -----------------------------
# 3) Base τ (Acct 0 starts with NO-SHARE across all assets)
#    Other rows = your provided matrix
# -----------------------------
base_tau = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],           # Account 0 (will flip k assets to 1 scenario by scenario)
    [1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,0,0,1,0],
    [0,0,0,1,0,1,0,1,1,0,1,1,1,1,1,1,0,0,0,0],
    [0,0,1,0,0,1,0,1,0,1,1,0,0,1,1,1,1,0,0,0],
    [1,1,0,0,0,0,1,1,1,0,0,1,1,1,1,0,1,0,0,0],
    [0,1,0,1,1,0,0,1,0,0,0,0,1,1,1,1,1,0,1,1],
    [1,1,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,1,0,1],
    [1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,1,0,1],
    [0,1,1,1,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0],
    [0,0,1,1,0,1,1,1,1,0,0,1,1,1,0,0,1,1,1,1]
], dtype=int)

# -----------------------------
# 4) Helper: build & solve a single τ scenario, then export everything we need later
# -----------------------------
def solve_and_export_for_tau(tau_mat: np.ndarray, scenario_tag: str):
    """
    tau_mat: N×d binary 0/1
    scenario_tag: e.g., "k00","k05","k20"
    Writes three CSVs:
      - account_performance_{scenario_tag}.csv
      - asset_trades_{scenario_tag}.csv
      - asset_pool_{scenario_tag}.csv
    """
    # --- Model ---
    model = xp.problem()

    # Vars
    w  = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
    wp = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
    wm = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
    w_pct = [[model.addVariable(lb=0.0, ub=1.0)     for j in range(d)] for n in range(N)]

    # Linearization of trades vs old holdings
    for n in range(N):
        for j in range(d):
            model.addConstraint(wp[n][j] - wm[n][j] == w[n][j] - old_holdings[n][j])

    # Budget constraints
    for n in range(N):
        model.addConstraint(xp.Sum(w[n][j] for j in range(d)) == budgets[n])
        for j in range(d):
            model.addConstraint(w[n][j] == budgets[n] * w_pct[n][j])
        model.addConstraint(xp.Sum(w_pct[n][j] for j in range(d)) == 1)

    # Sector constraints (same as your original)
    sector_idxs = {
        "T":  [tickers.index(t) for t in ["AAPL","MSFT","NVDA"]],
        "C":  [tickers.index(t) for t in ["GOOGL","META","DIS"]],
        "R":  [tickers.index(t) for t in ["AMZN","HD","TSLA"]],
        "H":  [tickers.index(t) for t in ["JNJ","UNH","PFE"]],
        "F":  [tickers.index(t) for t in ["JPM","BAC","V","MA"]],
        "S":  [tickers.index(t) for t in ["PG","KO"]],
        "E":  [tickers.index(t) for t in ["XOM","CVX"]],
    }
    model.addConstraint(xp.Sum(w[0][j] for j in sector_idxs["S"]+sector_idxs["H"]) >=  budgets[0]*0.40)
    model.addConstraint(xp.Sum(w[0][j] for j in sector_idxs["T"]+sector_idxs["R"]) <=  budgets[0]*0.25)

    model.addConstraint(xp.Sum(w[1][j] for j in sector_idxs["T"]) >=  budgets[1]*0.40)
    model.addConstraint(xp.Sum(w[1][j] for j in sector_idxs["C"]+[tickers.index("AMZN")]) >= budgets[1]*0.30)

    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["T"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["T"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["C"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["C"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["H"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["H"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["F"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["F"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["S"]+sector_idxs["E"]) >= budgets[2]*0.05)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["S"]+sector_idxs["E"]) <= budgets[2]*0.15)

    model.addConstraint(xp.Sum(w[3][j] for j in sector_idxs["S"]+sector_idxs["F"]+sector_idxs["E"]) >= budgets[3]*0.50)
    model.addConstraint(xp.Sum(w[3][j] for j in sector_idxs["F"]) >=  budgets[3]*0.20)

    model.addConstraint(xp.Sum(w[4][j] for j in sector_idxs["E"]) >=  budgets[4]*0.15)
    model.addConstraint(xp.Sum(w[4][j] for j in sector_idxs["F"]) >=  budgets[4]*0.25)

    model.addConstraint(xp.Sum(w[5][j] for j in sector_idxs["T"]) >=  budgets[5]*0.30)
    model.addConstraint(xp.Sum(w[5][j] for j in sector_idxs["H"]) >=  budgets[5]*0.20)
    model.addConstraint(xp.Sum(w[5][j] for j in sector_idxs["S"]+sector_idxs["R"]) <= budgets[5]*0.15)

    growth6 = sector_idxs["T"] + sector_idxs["C"] + sector_idxs["R"]
    model.addConstraint(xp.Sum(w[6][j] for j in growth6) <= budgets[6]*0.35)
    model.addConstraint(xp.Sum(w[6][j] for j in sector_idxs["E"]+sector_idxs["F"]+sector_idxs["S"]) >= budgets[6]*0.40)

    model.addConstraint(xp.Sum(w[7][j] for j in [tickers.index("AMZN"), tickers.index("TSLA")]) >= budgets[7]*0.30)
    model.addConstraint(xp.Sum(w[7][j] for j in sector_idxs["T"]+sector_idxs["F"]+sector_idxs["H"]) <= budgets[7]*0.50)

    growth8 = [tickers.index(t) for t in ["NVDA","TSLA","AAPL","MSFT"]]
    model.addConstraint(xp.Sum(w[8][j] for j in growth8) >= budgets[8]*0.50)

    model.addConstraint(xp.Sum(w[9][j] for j in sector_idxs["S"]+sector_idxs["E"]) >= budgets[9]*0.20)
    model.addConstraint(xp.Sum(w[9][j] for j in sector_idxs["T"]+sector_idxs["C"]+sector_idxs["R"]) >= budgets[9]*0.30)

    # Objective
    lin = xp.Sum(mu_list[n][j] * w[n][j] for n in range(N) for j in range(d))
    risk = xp.Sum(
        -0.5 * theta_list[n] * (budgets[n]**2) *
        xp.Sum(Sigma_list[n][i][j] * w_pct[n][i] * w_pct[n][j] for i in range(d) for j in range(d))
        for n in range(N)
    )

    # Pooled & independent impact
    tau_arr = np.asarray(tau_mat, dtype=int)
    T_plus  = [xp.Sum(tau_arr[k, j] * wp[k][j]  for k in range(N)) for j in range(d)]
    T_minus = [xp.Sum(tau_arr[k, j] * wm[k][j] for k in range(N)) for j in range(d)]

    quad_pp = xp.Sum(T_plus[i]  * xp.Sum(Omega_plus[i,  j] * T_plus[j]  for j in range(d)) for i in range(d))
    quad_mm = xp.Sum(T_minus[i] * xp.Sum(Omega_plus[i,  j] * T_minus[j] for j in range(d)) for i in range(d))
    cross_pm= xp.Sum(T_plus[i]  * xp.Sum(Omega_minus[i, j] * T_minus[j] for j in range(d)) for i in range(d))
    impact_pool = 0.5 * (quad_pp + quad_mm + 2 * cross_pm)

    impact_ind = 0
    for m in range(N):
        r_p = [ (1 - tau_arr[m, j]) * wp[m][j]  for j in range(d) ]
        r_m = [ (1 - tau_arr[m, j]) * wm[m][j] for j in range(d) ]
        quad_pp_m = xp.Sum(r_p[i] * xp.Sum(Omega_plus[i,  j] * r_p[j] for j in range(d)) for i in range(d))
        quad_mm_m = xp.Sum(r_m[i] * xp.Sum(Omega_plus[i,  j] * r_m[j] for j in range(d)) for i in range(d))
        cross_pm_m= xp.Sum(r_p[i] * xp.Sum(Omega_minus[i, j] * r_m[j] for j in range(d)) for i in range(d))
        impact_ind += 0.5 * (quad_pp_m + quad_mm_m + 2 * cross_pm_m)

    impact = - trading_days * (impact_pool + impact_ind)
    model.setObjective(lin + risk + impact, sense=xp.maximize)

    model.solve()

    # --- Pull solutions ---
    w_val  = np.zeros((N, d))
    wp_val = np.zeros((N, d))
    wm_val = np.zeros((N, d))
    for n in range(N):
        for j in range(d):
            w_val[n, j]  = model.getSolution(w[n][j])
            wp_val[n, j] = model.getSolution(wp[n][j])
            wm_val[n, j] = model.getSolution(wm[n][j])

    # --- Build summaries consistent with the model objective ---
    budgets_arr = np.asarray(budgets, dtype=float)
    w_pct_val   = w_val / budgets_arr[:, None]

    revenues = np.array([mu_list[n] @ w_val[n] for n in range(N)])
    risk_pen = np.array([
        -0.5 * theta_list[n] * (budgets_arr[n]**2) * (w_pct_val[n] @ Sigma_list[n] @ w_pct_val[n])
        for n in range(N)
    ])

    # pooled cost split per account (exact via gradient split)
    q_plus  = tau_arr * wp_val
    q_minus = tau_arr * wm_val
    Tplus   = q_plus.sum(axis=0)
    Tminus  = q_minus.sum(axis=0)

    term_lin = (
        (q_plus  @ Omega_plus  @ Tplus) +
        (q_minus @ Omega_plus  @ Tminus) +
        (q_plus  @ Omega_minus @ Tminus) +
        (q_minus @ Omega_minus @ Tplus)
    )
    self_quad = 0.5 * (
        np.einsum('ni,ij,nj->n', q_plus,  Omega_plus,  q_plus) +
        np.einsum('ni,ij,nj->n', q_minus, Omega_plus,  q_minus) +
        2.0 * np.einsum('ni,ij,nj->n', q_plus,  Omega_minus, q_minus)
    )
    C_pooled = term_lin - self_quad
    C_ind = np.zeros(N)
    for n in range(N):
        r_p = (1 - tau_arr[n]) * wp_val[n]
        r_m = (1 - tau_arr[n]) * wm_val[n]
        C_ind[n] = 0.5 * (
            r_p @ Omega_plus  @ r_p +
            r_m @ Omega_plus  @ r_m +
            2.0 * (r_p @ Omega_minus @ r_m)
        )

    txn_cost = trading_days * (C_pooled + C_ind)
    net_u    = revenues + risk_pen - txn_cost

    # --- (A) Per-account summary CSV ---
    acct_df = pd.DataFrame({
        "Revenue": revenues,
        "Risk Penalty": risk_pen,
        "Transaction Cost": txn_cost,
        "Net Utility": net_u
    }, index=[f"Acct {i}" for i in range(N)])

    # Account-level self-impact (sum_i s_{j,i}, using per-asset diag-only form, in ANNUAL dollars)
    Op_diag = np.diag(Omega_plus)
    Om_diag = np.diag(Omega_minus)
    s_asset = np.zeros((N, d))
    for n in range(N):
        for i in range(d):
            qpi = q_plus[n, i]
            qmi = q_minus[n, i]
            s_asset[n, i] = 0.5 * (qpi*qpi*Op_diag[i] + qmi*qmi*Op_diag[i] + 2.0*qpi*qmi*Om_diag[i])
    s_acct_annual = trading_days * s_asset.sum(axis=1)
    acct_df["SelfImpact (sum over assets, annual)"] = s_acct_annual

    acct_path = os.path.join(OUTDIR, f"account_performance_{scenario_tag}.csv")
    acct_df.to_csv(acct_path, float_format="%.6f")

    # --- (B) Asset-level details per account (for rebate weights later) ---
    long_rows = []
    for n in range(N):
        for i in range(d):
            long_rows.append({
                "Account": f"Acct {n}",
                "Asset": tickers[i],
                "i": i, "j": n,
                "tau_ij": int(tau_arr[n, i]),
                "w_plus":  wp_val[n, i],
                "w_minus": wm_val[n, i],
                "q_plus":  q_plus[n, i],
                "q_minus": q_minus[n, i],
                "s_self_daily_diag": s_asset[n, i],                      # daily
                "s_self_annual_diag": trading_days * s_asset[n, i]       # annual
            })
    asset_trades_df = pd.DataFrame(long_rows)
    trades_path = os.path.join(OUTDIR, f"asset_trades_{scenario_tag}.csv")
    asset_trades_df.to_csv(trades_path, index=False, float_format="%.6f")

    # --- (C) Per-asset pooled totals: rebate fund A_i (diag-only), participants P_i ---
    pool_rows = []
    for i in range(d):
        A_i_daily = 0.5 * (
            Tplus[i]*Tplus[i]*Op_diag[i] +
            Tminus[i]*Tminus[i]*Op_diag[i] +
            2.0*Tplus[i]*Tminus[i]*Om_diag[i]
        )
        part_idx = [f"Acct {n}" for n in range(N) if tau_arr[n, i] == 1]
        pool_rows.append({
            "Asset": tickers[i],
            "i": i,
            "T_plus":  Tplus[i],
            "T_minus": Tminus[i],
            "A_i_daily_diag": A_i_daily,
            "A_i_annual_diag": trading_days * A_i_daily,
            "participants_count": len(part_idx),
            "participants": ";".join(part_idx),
            "Omega_plus_ii": Op_diag[i],
            "Omega_minus_ii": Om_diag[i]
        })
    asset_pool_df = pd.DataFrame(pool_rows)
    pool_path = os.path.join(OUTDIR, f"asset_pool_{scenario_tag}.csv")
    asset_pool_df.to_csv(pool_path, index=False, float_format="%.6f")

    # Return small header for the scenarios index
    acct0_net = float(acct_df.loc["Acct 0", "Net Utility"])
    acct0_tc  = float(acct_df.loc["Acct 0", "Transaction Cost"])
    return acct0_net, acct0_tc

# -----------------------------
# 5) Export invariant matrices (once)
# -----------------------------
if SAVE_MATRICES_ONCE:
    pd.DataFrame(Omega_plus,  index=tickers, columns=tickers).to_csv(os.path.join(OUTDIR, "Omega_plus.csv"),  float_format="%.12g")
    pd.DataFrame(Omega_minus, index=tickers, columns=tickers).to_csv(os.path.join(OUTDIR, "Omega_minus.csv"), float_format="%.12g")
    pd.Series(tickers).to_csv(os.path.join(OUTDIR, "tickers.csv"), index=False)
    pd.Series(budgets, index=[f"Acct {i}" for i in range(N)]).to_csv(os.path.join(OUTDIR, "budgets.csv"), header=["budget"], float_format="%.6f")
    pd.DataFrame(corr, index=tickers, columns=tickers).to_csv(os.path.join(OUTDIR, "corr.csv"), float_format="%.6f")

# -----------------------------
# 6) Batch 20 τ scenarios: Acct 0 shares k=0..20 assets (ticker order)
# -----------------------------
share_order = list(range(d))  # order in which Acct 0 flips assets to 1
scn_records = []

for k in range(0, d+1):  # 0..20
    tau_k = base_tau.copy()
    if k > 0:
        tau_k[0, share_order[:k]] = 1
    tag = f"tau_k{k:02d}"
    acct0_net, acct0_tc = solve_and_export_for_tau(tau_k, tag)
    scn_records.append({
        "scenario": tag,
        "k_shared_acct0": k,
        "acct0_net_utility": acct0_net,
        "acct0_transaction_cost": acct0_tc,
        "acct0_shared_assets": ",".join([tickers[i] for i in share_order[:k]])
    })
    print(f"[done] {tag}: Acct0 k={k}, Net={acct0_net:,.2f}, Cost={acct0_tc:,.2f}")

pd.DataFrame(scn_records).to_csv(os.path.join(OUTDIR, "scenarios_index.csv"), index=False)

print("\nAll scenarios finished.")
print(f"Outputs are in: {os.path.abspath(OUTDIR)}")
print("Per-scenario files:")
print("  - account_performance_tau_kXX.csv")
print("  - asset_trades_tau_kXX.csv")
print("  - asset_pool_tau_kXX.csv")
print("Master index: scenarios_index.csv; Invariants: Omega_plus.csv, Omega_minus.csv, tickers.csv, budgets.csv, corr.csv")


FICO Xpress v9.5.0, Hyper, solve started 20:40:00, Aug 18, 2025
Heap usage: 1335KB (peak 2195KB, 85KB system)
Maximizing QP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
       448 rows          800 cols         1527 elements
     41032 qobjelem
Presolved problem has:
       233 rows          600 cols          910 elements
      6416 qobjelem
Presolve finished in 0 seconds
Heap usage: 2048KB (peak 2488KB, 85KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.50e+06] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.50e+06] / [ 8.24e+01,  1.50e+06]
  Objective      [min,max] : [ 1.31e-02,  7.54e-01] / [ 1.31e-02,  7.54e-01]
  Quadratic      [min,max] : [ 2.47e-11,  3.60e+06] / [ 1.05e-09,  3.50e-06]
Autoscaling applied standard scaling

Using AVX2 support
Cor

In [3]:
# -*- coding: utf-8 -*-
"""
InfoPool (single-period) — batch 21 τ scenarios where:
- Account 0 NEVER shares (row-0 of τ is all zeros for every scenario);
- Other accounts (1..9) gradually start sharing BY COLUMNS:
  scenario k sets the first k columns to 1 for rows 1..9 (row 0 stays 0).
All other calculations remain unchanged.

Exports per scenario:
  - account_performance_taucols_kXX.csv  (all accounts)
  - asset_trades_taucols_kXX.csv         (per-account, per-asset details)
  - asset_pool_taucols_kXX.csv           (per-asset pool totals & participants)
  - tau_matrix_taucols_kXX.csv           (the full τ matrix for traceability)

Also exports (once):
  - Omega_plus.csv, Omega_minus.csv, tickers.csv, budgets.csv, corr.csv

And a master index:
  - scenarios_index.csv  (k, which columns flipped, per-account shared-count, Acct0 net/cost)

This script reuses your Ω-construction, investor prefs, constraints, and objective.
"""

import os
import hashlib
import numpy as np
import pandas as pd
import yfinance as yf
import xpress as xp

# -----------------------------
# 0) Paths & switches
# -----------------------------
OUTDIR = "infopool_batch_outputs_colsweep"
os.makedirs(OUTDIR, exist_ok=True)

SAVE_MATRICES_ONCE = True
ANNUAL_TRADING_DAYS = 252

# -----------------------------
# 1) Market data & Ω-construction (same as your earlier setup)
# -----------------------------
xp.init("C:/Users/s2667242/AppData/Local/anaconda3/Lib/site-packages/xpress/license/xpauth.xpr")

tickers = [
    "AAPL","MSFT","GOOGL","AMZN","META","TSLA","JNJ","JPM","V","PG",
    "NVDA","DIS","UNH","HD","MA","PFE","BAC","KO","XOM","CVX"
]
d = len(tickers)

start_date = "2020-01-01"
end_date   = "2025-07-24"

data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False, progress=False)
prices_adj = data["Adj Close"]
prices_raw = data["Close"]
volumes    = data["Volume"]

returns = prices_adj.pct_change().dropna()
mean_daily_returns = returns.mean()
cov_matrix         = returns.cov()

sigma_all  = returns.std(axis=0).values
up_returns   = returns.clip(lower=0)
down_returns = (-returns).clip(lower=0)
sigma_plus   = up_returns.std(axis=0).values
sigma_minus  = down_returns.std(axis=0).values

dollar_vol = (prices_raw.reindex_like(volumes) * volumes).loc[returns.index]
Vbar = dollar_vol.mean(axis=0).values

def winsorize_ser(s: pd.Series, q=0.995):
    lo, hi = s.quantile(1-q), s.quantile(q)
    return s.clip(lower=lo, upper=hi)

def estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_vec, side="both"):
    px_raw = prices_raw.loc[returns.index]
    vol    = volumes.loc[returns.index]
    dollar_vol = (px_raw * vol).replace(0, np.nan)
    illiq = (returns.abs() / dollar_vol).replace([np.inf, -np.inf], np.nan)
    if side == "plus":
        mask = (returns > 0)
    elif side == "minus":
        mask = (returns < 0)
    else:
        mask = pd.DataFrame(True, index=returns.index, columns=returns.columns)
    illiq_med = winsorize_ser(illiq.where(mask).median(axis=0, skipna=True), q=0.995)
    Vbar_local = (px_raw * vol).mean(axis=0)
    sigma = pd.Series(sigma_vec, index=returns.columns)
    kappa = (illiq_med * Vbar_local / sigma).replace([np.inf, -np.inf], np.nan)
    return kappa.clip(lower=0.01, upper=100.0).values

def gershgorin_delta_max(gamma: np.ndarray, corr: np.ndarray):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    row_sum_off = np.where(row_sum_off <= 1e-12, np.inf, row_sum_off)
    return float(np.min(gamma / row_sum_off))

def delta_from_ratio(gamma: np.ndarray, corr: np.ndarray, rho=0.3, cap_ratio=0.9):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    factor = row_sum_off / gamma
    mean_factor = np.mean(factor[np.isfinite(factor) & (factor > 0)])
    delta_star = rho / mean_factor if (mean_factor is not None and mean_factor > 0) else 0.0
    delta_max  = gershgorin_delta_max(gamma, corr)
    return float(min(delta_star, cap_ratio * delta_max)), float(delta_max)

kappa_plus_hat  = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_plus,  side="plus")
kappa_minus_hat = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_minus, side="minus")
gamma_plus  = kappa_plus_hat  * sigma_plus  / Vbar
gamma_minus = kappa_minus_hat * sigma_minus / Vbar

corr = cov_matrix.values / np.outer(sigma_all, sigma_all)
delta_plus,  _ = delta_from_ratio(gamma_plus,  corr, rho=0.3, cap_ratio=0.9)
delta_minus, _ = delta_from_ratio(gamma_minus, corr, rho=0.3, cap_ratio=0.9)

Gp = np.sqrt(np.outer(gamma_plus,  gamma_plus))
Gm = np.sqrt(np.outer(gamma_minus, gamma_minus))
Omega_plus  = np.diag(gamma_plus)  + delta_plus  * (Gp * corr)
Omega_minus = np.diag(gamma_minus) + delta_minus * (Gm * corr)

xi = 0.10
target_bps = 100
c = target_bps / 1e4
s_cand_p = 2*c / np.maximum(gamma_plus  * xi * Vbar, 1e-30)
s_cand_m = 2*c / np.maximum(gamma_minus * xi * Vbar, 1e-30)
s_omega  = float(np.median(np.concatenate([s_cand_p, s_cand_m])))
Omega_plus  = s_omega * Omega_plus
Omega_minus = s_omega * Omega_minus

# -----------------------------
# 2) Investor prefs & budgets (same as your model)
# -----------------------------
N = 10
trading_days = ANNUAL_TRADING_DAYS

mu_vec      = mean_daily_returns.values
Sigma_mat   = cov_matrix.values
mu_annual   = mu_vec * trading_days
Sigma_annual= Sigma_mat * trading_days
mu_list     = [mu_annual.copy()   for _ in range(N)]
Sigma_list  = [Sigma_annual.copy() for _ in range(N)]

gamma_list = [
    0.49671415, 0.64768854, 0.76743473, 0.54256004, 0.24196227,
    0.31424733, 0.06752820, 0.11092259, 0.37569802, 0.82254491
]
budgets = [1e6, 5e5, 8e5, 1.2e6, 6e5, 9e5, 7e5, 4e5, 1.5e6, 1e6]

alpha_risk = 0.5
w_pct0 = np.full((N, d), 1.0/d)
risk_proto = np.array([
    0.5 * gamma_list[n] * (budgets[n]**2) * (w_pct0[n] @ Sigma_list[n] @ w_pct0[n])
    for n in range(N)
])
rev_est = np.array([budgets[n] * (mu_list[n] @ w_pct0[n]) for n in range(N)])
s = (alpha_risk * max(rev_est.mean(), 1e-12)) / max(risk_proto.mean(), 1e-12)
theta_list = (s * np.array(gamma_list)).tolist()

# Initial holdings (deterministic)
INIT_HOLDINGS_PATH = os.path.join(OUTDIR, "initial_holdings.csv")
INIT_SEED = 12345
def _dirichlet_like_from_hash(tickers, budgets, seed=INIT_SEED):
    Nn, dd = len(budgets), len(tickers)
    W = np.zeros((Nn, dd), dtype=float)
    for n in range(Nn):
        e = np.empty(dd, dtype=float)
        for j, t in enumerate(tickers):
            key = f"{seed}|acct={n}|ticker={t}".encode()
            h = hashlib.sha256(key).digest()
            u64 = int.from_bytes(h[:8], "big")
            u = (u64 + 0.5) / 2**64
            u = min(max(u, 1e-12), 1-1e-12)
            e[j] = -np.log(u)
        w = e / e.sum()
        W[n, :] = w
    B = np.asarray(budgets, dtype=float)[:, None]
    return W * B

def _load_or_make_initial_holdings(tickers, budgets, path=INIT_HOLDINGS_PATH, seed=INIT_SEED):
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0)
        if (df.shape[0]==len(budgets)) and (df.shape[1]==len(tickers)) and (list(df.columns)==list(tickers)):
            return df.values
    arr = _dirichlet_like_from_hash(tickers, budgets, seed=seed)
    pd.DataFrame(arr, index=[f"Acct {i}" for i in range(len(budgets))], columns=tickers)\
      .to_csv(path, float_format="%.6f")
    return arr

old_holdings = _load_or_make_initial_holdings(tickers, budgets)

# -----------------------------
# 3) Base τ (from your earlier matrix, BUT row-0 will be overridden to zeros)
# -----------------------------
base_tau = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],  # row-0 stays 0 forever (Acct 0 never shares)
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
], dtype=int)

# -----------------------------
# 4) Solve one τ and export all needed tables
# -----------------------------
def solve_and_export_for_tau(tau_mat: np.ndarray, scenario_tag: str):
    model = xp.problem()

    w  = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
    wp = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
    wm = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
    w_pct = [[model.addVariable(lb=0.0, ub=1.0)     for j in range(d)] for n in range(N)]

    for n in range(N):
        for j in range(d):
            model.addConstraint(wp[n][j] - wm[n][j] == w[n][j] - old_holdings[n][j])

    for n in range(N):
        model.addConstraint(xp.Sum(w[n][j] for j in range(d)) == budgets[n])
        for j in range(d):
            model.addConstraint(w[n][j] == budgets[n] * w_pct[n][j])
        model.addConstraint(xp.Sum(w_pct[n][j] for j in range(d)) == 1)

    sector_idxs = {
        "T":  [tickers.index(t) for t in ["AAPL","MSFT","NVDA"]],
        "C":  [tickers.index(t) for t in ["GOOGL","META","DIS"]],
        "R":  [tickers.index(t) for t in ["AMZN","HD","TSLA"]],
        "H":  [tickers.index(t) for t in ["JNJ","UNH","PFE"]],
        "F":  [tickers.index(t) for t in ["JPM","BAC","V","MA"]],
        "S":  [tickers.index(t) for t in ["PG","KO"]],
        "E":  [tickers.index(t) for t in ["XOM","CVX"]],
    }
    model.addConstraint(xp.Sum(w[0][j] for j in sector_idxs["S"]+sector_idxs["H"]) >=  budgets[0]*0.40)
    model.addConstraint(xp.Sum(w[0][j] for j in sector_idxs["T"]+sector_idxs["R"]) <=  budgets[0]*0.25)

    model.addConstraint(xp.Sum(w[1][j] for j in sector_idxs["T"]) >=  budgets[1]*0.40)
    model.addConstraint(xp.Sum(w[1][j] for j in sector_idxs["C"]+[tickers.index("AMZN")]) >= budgets[1]*0.30)

    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["T"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["T"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["C"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["C"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["H"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["H"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["F"]) >=  budgets[2]*0.10)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["F"]) <=  budgets[2]*0.25)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["S"]+sector_idxs["E"]) >= budgets[2]*0.05)
    model.addConstraint(xp.Sum(w[2][j] for j in sector_idxs["S"]+sector_idxs["E"]) <= budgets[2]*0.15)

    model.addConstraint(xp.Sum(w[3][j] for j in sector_idxs["S"]+sector_idxs["F"]+sector_idxs["E"]) >= budgets[3]*0.50)
    model.addConstraint(xp.Sum(w[3][j] for j in sector_idxs["F"]) >=  budgets[3]*0.20)

    model.addConstraint(xp.Sum(w[4][j] for j in sector_idxs["E"]) >=  budgets[4]*0.15)
    model.addConstraint(xp.Sum(w[4][j] for j in sector_idxs["F"]) >=  budgets[4]*0.25)

    model.addConstraint(xp.Sum(w[5][j] for j in sector_idxs["T"]) >=  budgets[5]*0.30)
    model.addConstraint(xp.Sum(w[5][j] for j in sector_idxs["H"]) >=  budgets[5]*0.20)
    model.addConstraint(xp.Sum(w[5][j] for j in sector_idxs["S"]+sector_idxs["R"]) <= budgets[5]*0.15)

    growth6 = sector_idxs["T"] + sector_idxs["C"] + sector_idxs["R"]
    model.addConstraint(xp.Sum(w[6][j] for j in growth6) <= budgets[6]*0.35)
    model.addConstraint(xp.Sum(w[6][j] for j in sector_idxs["E"]+sector_idxs["F"]+sector_idxs["S"]) >= budgets[6]*0.40)

    model.addConstraint(xp.Sum(w[7][j] for j in [tickers.index("AMZN"), tickers.index("TSLA")]) >= budgets[7]*0.30)
    model.addConstraint(xp.Sum(w[7][j] for j in sector_idxs["T"]+sector_idxs["F"]+sector_idxs["H"]) <= budgets[7]*0.50)

    growth8 = [tickers.index(t) for t in ["NVDA","TSLA","AAPL","MSFT"]]
    model.addConstraint(xp.Sum(w[8][j] for j in growth8) >= budgets[8]*0.50)

    model.addConstraint(xp.Sum(w[9][j] for j in sector_idxs["S"]+sector_idxs["E"]) >= budgets[9]*0.20)
    model.addConstraint(xp.Sum(w[9][j] for j in sector_idxs["T"]+sector_idxs["C"]+sector_idxs["R"]) >= budgets[9]*0.30)

    lin = xp.Sum(mu_list[n][j] * w[n][j] for n in range(N) for j in range(d))
    risk = xp.Sum(
        -0.5 * theta_list[n] * (budgets[n]**2) *
        xp.Sum(Sigma_list[n][i][j] * w_pct[n][i] * w_pct[n][j] for i in range(d) for j in range(d))
        for n in range(N)
    )

    tau_arr = np.asarray(tau_mat, dtype=int)
    T_plus  = [xp.Sum(tau_arr[k, j] * wp[k][j]  for k in range(N)) for j in range(d)]
    T_minus = [xp.Sum(tau_arr[k, j] * wm[k][j] for k in range(N)) for j in range(d)]

    quad_pp = xp.Sum(T_plus[i]  * xp.Sum(Omega_plus[i,  j] * T_plus[j]  for j in range(d)) for i in range(d))
    quad_mm = xp.Sum(T_minus[i] * xp.Sum(Omega_plus[i,  j] * T_minus[j] for j in range(d)) for i in range(d))
    cross_pm= xp.Sum(T_plus[i]  * xp.Sum(Omega_minus[i, j] * T_minus[j] for j in range(d)) for i in range(d))
    impact_pool = 0.5 * (quad_pp + quad_mm + 2 * cross_pm)

    impact_ind = 0
    for m in range(N):
        r_p = [ (1 - tau_arr[m, j]) * wp[m][j]  for j in range(d) ]
        r_m = [ (1 - tau_arr[m, j]) * wm[m][j] for j in range(d) ]
        quad_pp_m = xp.Sum(r_p[i] * xp.Sum(Omega_plus[i,  j] * r_p[j] for j in range(d)) for i in range(d))
        quad_mm_m = xp.Sum(r_m[i] * xp.Sum(Omega_plus[i,  j] * r_m[j] for j in range(d)) for i in range(d))
        cross_pm_m= xp.Sum(r_p[i] * xp.Sum(Omega_minus[i, j] * r_m[j] for j in range(d)) for i in range(d))
        impact_ind += 0.5 * (quad_pp_m + quad_mm_m + 2 * cross_pm_m)

    impact = - trading_days * (impact_pool + impact_ind)
    model.setObjective(lin + risk + impact, sense=xp.maximize)

    model.solve()

    w_val  = np.zeros((N, d))
    wp_val = np.zeros((N, d))
    wm_val = np.zeros((N, d))
    for n in range(N):
        for j in range(d):
            w_val[n, j]  = model.getSolution(w[n][j])
            wp_val[n, j] = model.getSolution(wp[n][j])
            wm_val[n, j] = model.getSolution(wm[n][j])

    budgets_arr = np.asarray(budgets, dtype=float)
    w_pct_val   = w_val / budgets_arr[:, None]

    revenues = np.array([mu_list[n] @ w_val[n] for n in range(N)])
    risk_pen = np.array([
        -0.5 * theta_list[n] * (budgets_arr[n]**2) * (w_pct_val[n] @ Sigma_list[n] @ w_pct_val[n])
        for n in range(N)
    ])

    q_plus  = tau_arr * wp_val
    q_minus = tau_arr * wm_val
    Tplus   = q_plus.sum(axis=0)
    Tminus  = q_minus.sum(axis=0)

    term_lin = (
        (q_plus  @ Omega_plus  @ Tplus) +
        (q_minus @ Omega_plus  @ Tminus) +
        (q_plus  @ Omega_minus @ Tminus) +
        (q_minus @ Omega_minus @ Tplus)
    )
    self_quad = 0.5 * (
        np.einsum('ni,ij,nj->n', q_plus,  Omega_plus,  q_plus) +
        np.einsum('ni,ij,nj->n', q_minus, Omega_plus,  q_minus) +
        2.0 * np.einsum('ni,ij,nj->n', q_plus,  Omega_minus, q_minus)
    )
    C_pooled = term_lin - self_quad

    C_ind = np.zeros(N)
    for n in range(N):
        r_p = (1 - tau_arr[n]) * wp_val[n]
        r_m = (1 - tau_arr[n]) * wm_val[n]
        C_ind[n] = 0.5 * (
            r_p @ Omega_plus  @ r_p +
            r_m @ Omega_plus  @ r_m +
            2.0 * (r_p @ Omega_minus @ r_m)
        )

    txn_cost = trading_days * (C_pooled + C_ind)
    net_u    = revenues + risk_pen - txn_cost

    # (A) Per-account summary
    acct_df = pd.DataFrame({
        "Revenue": revenues,
        "Risk Penalty": risk_pen,
        "Transaction Cost": txn_cost,
        "Net Utility":       net_u
    }, index=[f"Acct {i}" for i in range(N)])

    # Per-account self-impact (sum over assets), diag-only, annual
    Op_diag = np.diag(Omega_plus)
    Om_diag = np.diag(Omega_minus)
    s_asset_daily = np.zeros((N, d))
    for n in range(N):
        for i in range(d):
            qpi = q_plus[n, i]
            qmi = q_minus[n, i]
            s_asset_daily[n, i] = 0.5 * (qpi*qpi*Op_diag[i] + qmi*qmi*Op_diag[i] + 2.0*qpi*qmi*Om_diag[i])
    s_acct_annual = trading_days * s_asset_daily.sum(axis=1)
    acct_df["SelfImpact (sum over assets, annual)"] = s_acct_annual

    acct_path = os.path.join(OUTDIR, f"account_performance_{scenario_tag}.csv")
    acct_df.to_csv(acct_path, float_format="%.6f")

    # (B) Asset-level details per account
    long_rows = []
    for n in range(N):
        for i in range(d):
            long_rows.append({
                "Account": f"Acct {n}",
                "Asset": tickers[i],
                "i": i, "j": n,
                "tau_ij": int(tau_arr[n, i]),
                "w_plus":  wp_val[n, i],
                "w_minus": wm_val[n, i],
                "q_plus":  q_plus[n, i],
                "q_minus": q_minus[n, i],
                "s_self_daily_diag":  s_asset_daily[n, i],
                "s_self_annual_diag": trading_days * s_asset_daily[n, i]
            })
    trades_path = os.path.join(OUTDIR, f"asset_trades_{scenario_tag}.csv")
    pd.DataFrame(long_rows).to_csv(trades_path, index=False, float_format="%.6f")

    # (C) Per-asset pool info
    pool_rows = []
    for i in range(d):
        A_i_daily = 0.5 * (
            Tplus[i]*Tplus[i]*Op_diag[i] +
            Tminus[i]*Tminus[i]*Op_diag[i] +
            2.0*Tplus[i]*Tminus[i]*Om_diag[i]
        )
        part_idx = [f"Acct {n}" for n in range(N) if tau_arr[n, i] == 1]
        pool_rows.append({
            "Asset": tickers[i],
            "i": i,
            "T_plus":  Tplus[i],
            "T_minus": Tminus[i],
            "A_i_daily_diag":  A_i_daily,
            "A_i_annual_diag": trading_days * A_i_daily,
            "participants_count": len(part_idx),
            "participants": ";".join(part_idx),
            "Omega_plus_ii":  Op_diag[i],
            "Omega_minus_ii": Om_diag[i]
        })
    pool_path = os.path.join(OUTDIR, f"asset_pool_{scenario_tag}.csv")
    pd.DataFrame(pool_rows).to_csv(pool_path, index=False, float_format="%.6f")

    # (D) τ matrix for traceability
    tau_df = pd.DataFrame(tau_arr, index=[f"Acct {i}" for i in range(N)], columns=tickers)
    tau_mat_path = os.path.join(OUTDIR, f"tau_matrix_{scenario_tag}.csv")
    tau_df.to_csv(tau_mat_path, float_format="%.0f")

    acct0_net = float(acct_df.loc["Acct 0", "Net Utility"])
    acct0_tc  = float(acct_df.loc["Acct 0", "Transaction Cost"])
    shared_counts = tau_arr.sum(axis=1).tolist()
    return acct0_net, acct0_tc, shared_counts

# -----------------------------
# 5) Export invariant matrices once
# -----------------------------
if SAVE_MATRICES_ONCE:
    pd.DataFrame(Omega_plus,  index=tickers, columns=tickers).to_csv(os.path.join(OUTDIR, "Omega_plus.csv"),  float_format="%.12g")
    pd.DataFrame(Omega_minus, index=tickers, columns=tickers).to_csv(os.path.join(OUTDIR, "Omega_minus.csv"), float_format="%.12g")
    pd.Series(tickers).to_csv(os.path.join(OUTDIR, "tickers.csv"), index=False)
    pd.Series(budgets, index=[f"Acct {i}" for i in range(N)]).to_csv(os.path.join(OUTDIR, "budgets.csv"), header=["budget"], float_format="%.6f")
    pd.DataFrame(corr, index=tickers, columns=tickers).to_csv(os.path.join(OUTDIR, "corr.csv"), float_format="%.6f")

# -----------------------------
# 6) Batch 21 τ scenarios: flip columns for accounts 1..9 (Acct 0 fixed at 0)
# -----------------------------
share_order_cols = list(range(d))  # flip columns in ticker order
scn_records = []

for k in range(0, d+1):  # k = 0..20
    tau_k = base_tau.copy()
    # Row-0 must be all zeros (Acct 0 never shares)
    tau_k[0, :] = 1
    # For rows 1..9, set the first k columns to 1 (gradually increasing shared assets across all other accounts)
    if k > 0:
        tau_k[1:, share_order_cols[:k]] = 1

    tag = f"taucols_k{k:02d}"

    acct0_net, acct0_tc, shared_counts = solve_and_export_for_tau(tau_k, tag)

    # Build scenario index entry
    rec = {
        "scenario": tag,
        "k_shared_cols_for_others": k,  # how many columns (assets) are fully shared by accounts 1..9
        "flipped_assets": ",".join([tickers[i] for i in share_order_cols[:k]]),
        "acct0_net_utility": acct0_net,
        "acct0_transaction_cost": acct0_tc,
    }
    # per-account shared counts in this scenario
    for n in range(N):
        rec[f"shared_count_Acct{n}"] = shared_counts[n]
    scn_records.append(rec)

    print(f"[done] {tag}: k={k} flipped cols, Acct0 Net={acct0_net:,.2f}, Cost={acct0_tc:,.2f}")

# Master index
pd.DataFrame(scn_records).to_csv(os.path.join(OUTDIR, "scenarios_index.csv"), index=False)

print("\nAll scenarios finished.")
print(f"Outputs are in: {os.path.abspath(OUTDIR)}")
print("Per-scenario files:")
print("  - account_performance_taucols_kXX.csv")
print("  - asset_trades_taucols_kXX.csv")
print("  - asset_pool_taucols_kXX.csv")
print("  - tau_matrix_taucols_kXX.csv")
print("Master index: scenarios_index.csv; Invariants: Omega_plus.csv, Omega_minus.csv, tickers.csv, budgets.csv, corr.csv")


FICO Xpress v9.5.0, Hyper, solve started 0:36:15, Aug 19, 2025
Heap usage: 1006KB (peak 1499KB, 182KB system)
Maximizing QP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
       448 rows          800 cols         1527 elements
     20000 qobjelem
Presolved problem has:
       233 rows          600 cols          910 elements
      4880 qobjelem
Presolve finished in 0 seconds
Heap usage: 1391KB (peak 1695KB, 182KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.50e+06] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.50e+06] / [ 8.24e+01,  1.50e+06]
  Objective      [min,max] : [ 1.31e-02,  7.54e-01] / [ 1.31e-02,  7.54e-01]
  Quadratic      [min,max] : [ 2.47e-11,  3.60e+06] / [ 1.05e-09,  3.50e-06]
Autoscaling applied standard scaling

Using AVX2 support
Co

RuntimeError: Xpress global environment not available. If you're re-running in the SAME Python session, remove/guard xp.init() or restart the kernel.